In [1]:
# Pradėta 2024-11-21_ObjectProperties.ipynb
# Tęsinys. Čia 2024-11-27_varle_lt.ipynb

import pandas as pd
import time
import sqlite3

from bs4 import BeautifulSoup

# Selenium
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver

from my_lib.properties_of import properties_of
from my_lib.my_db_functions import save_df_to_sqlite

print('Bibliotekos importuotos.')

Bibliotekos importuotos.


In [ ]:
print('WebDriver started.',end=' ')
item_type = 'saldytuvai' # 'dronai', 'televizoriai'

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = f'https://www.varle.lt/{item_type}/'
driver.get(url)
time.sleep(3)
source = driver.page_source

print(f'Prskaityta {url}.')

WebDriver started. Prskaityta https://www.varle.lt/saldytuvai/.


In [14]:
bs = BeautifulSoup(source,'html.parser')

pages_numbers = bs.find('div',{'class':'PAGINATION'}).find_all('li',{'class':'number'})
total_pages = int(pages_numbers[-1].text.strip())

print(f'Total pages: {total_pages}')

items_df = pd.DataFrame(columns=('id','price','url','price_in_cents')) # šios programos rezultatas

Total pages: 33


In [ ]:
# Parsinamas pirmasis prekių sąrašo puslapis.

items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
for item_block in items_blocks:

    item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
    item_url = 'https://www.varle.lt'+ item_href
    item_id = item_href.split('-')[-1].split('.')[0]

    price_container = item_block.find('div',{'class':'price-container'})
    price_value = price_container.find('span',{'class':'price-value'}).getText()
    item_price = ''.join(price_value.split())

    items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

print(len(items_df))

38


In [ ]:
# Perskaitomi kiti prekių sąrašo puslapiai

for page_number in range(2,total_pages+1):

    url = f'https://www.varle.lt/{item_type}/?p={page_number}'
    driver.get(url)
    time.sleep(3)
    source = driver.page_source
    print(f'Prskaityta {url}')

    bs = BeautifulSoup(source,'html.parser')

    items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
    for item_block in items_blocks:
        item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
        item_url = 'https://www.varle.lt'+item_href
        item_id = item_href.split('-')[-1].split('.')[0]

        price_container = item_block.find('div',{'class':'price-container'})
        price_value = price_container.find('span',{'class':'price-value'}).getText()
        item_price = ''.join(price_value.split())

        items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

    items_total = len(items_df)
    items_df.drop_duplicates(subset='id',keep='first',inplace=True)
    print(f'Get {items_total} items. Unique {len(items_df)} items.')

# WebDriver stop!
driver.close()


Prskaityta https://www.varle.lt/saldytuvai/?p=2
Get 76 items. Unique 76 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=3
Get 114 items. Unique 114 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=4
Get 152 items. Unique 152 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=5
Get 190 items. Unique 190 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=6
Get 228 items. Unique 228 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=7
Get 266 items. Unique 266 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=8
Get 304 items. Unique 304 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=9
Get 342 items. Unique 342 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=10
Get 380 items. Unique 380 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=11
Get 418 items. Unique 418 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=12
Get 456 items. Unique 456 items.
Prskaityta https://www.varle.lt/saldytuvai/?p=13
Get 494 items. Unique 494 items.
Prskaityta https://www.var

In [7]:
# Patikrinimas: skaičiai skiriasi jei yra dublikatai.
grupuotas_df=items_df.groupby('id')
print(len(grupuotas_df),len(items_df))

119 119


In [17]:
import re

def price_in_cents(price:str) -> int:
    # Pašalina visus simbolius, išskyrus skaičius, kablelį ir tašką:
    cleaned = re.sub(r'[^\d.,]', '', price).replace(',', '.').replace(' ', '')
    return int(float(cleaned)*100)

items_df['price_in_cents'] = items_df['price'].apply(price_in_cents)


In [18]:
items_df

,id,price,url,price_in_cents
0,36822303,219€,https://www.varle.lt/saldytuvai/saldytuvas-sha...,21900
1,37859978,170€,https://www.varle.lt/saldytuvai/saldytuvas-his...,17000
2,28786311,777€,https://www.varle.lt/saldytuvai/imontuojamasis...,77700
3,15912426,450€,https://www.varle.lt/saldytuvai/saldytuvas-bos...,45000
4,31802747,450€,https://www.varle.lt/saldytuvai/saldytuvas-hai...,45000
...,...,...,...,...
1243,39312671,809.99€,https://www.varle.lt/saldytuvai/saldytuvas-sam...,80999
1244,34279221,809.99€,https://www.varle.lt/saldytuvai/saldytuvas-sam...,80999
1245,40216925,950.03€,https://www.varle.lt/saldytuvai/siemens-siemen...,95003
1246,39349057,235.99€,https://www.varle.lt/saldytuvai/saldytuvas-sha...,23599


In [19]:
# Save to SQLite3
save_df_to_sqlite(items_df,"./my_scrap_working_data.db",table_name=item_type)

Lentelė 'saldytuvai' sėkmingai įrašyta į DB.
